In [14]:
import pandas as pd
import numpy as np
import sklearn
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split as tts
import statsmodels.formula.api as smf
import nltk
sns.set
%matplotlib inline

In [15]:
df1 = pd.read_csv('13300.csv') ## Hent DataFrame
text = df1['text'] # Ekstraher alle tekster til en liste

tekster = text[0:13300] # sætter et interval blandt tekster til behandling
paragraffs = [] # holdinglist til alle paragraffer (resultatet bliver en liste i en liste)
### looper over alle tekster.
for i in tekster:
    try:
        val = nltk.tokenize.line_tokenize(i).lower() #tokenizer på linjeskift(paragraffer) 
        paragraffs.append(val) 
    except (AttributeError) as err: ## printer Error, som value, når der ikke forekommer en tekst
        val = 'Error'               ## Vi printer Error, som value, for at beholde det rette index
        paragraffs.append(val)      ##

df1['paragraffs'] = paragraffs # paragrafferne tilføjes til DF, som en liste med strings.
# der laves en ny dataframe, hvor listerne paragraffer foldes ud.
df_para = df1.apply(lambda x: pd.Series(x['paragraffs']), axis=1).stack().reset_index(level=1, drop=True)
df_para.name = 'paragraffer'
# dataframes merges til en endelig datafram, hvor alle paragraffer er foldet ud. 
df1_p = df1.drop('paragraffs', axis=1).join(df_para)
df1_p = df1_p.drop('Unnamed: 0', axis=1)

In [16]:
# Tilføj emner til dataframe
emner = ['international politik','økonomi & velfærd','ret & politi','familie & børn','uddannelse','kultur & religion','køn & ligestilling','medier, kommunikation & it','miljø & klima','flygtninge & intengrationspolitik','andet','ignore']
for i in emner:
    df1_p[i] = pd.Series()

In [17]:
# generate 2000 random rows for sampling
df1_p_sample = df1_p.sample(n=1999, random_state=1) 
# split the samples into 4
part1_sample1 = df1_p_sample[0:499]
part1_sample2 = df1_p_sample[500:999]
part1_sample3 = df1_p_sample[1000:1499]
part1_sample4 = df1_p_sample[1500:1999]
# concatenate all combinations of 3 parts of the sample
f1 = pd.concat([pd.concat([part1_sample2, part1_sample3]), part1_sample4])
f2 = pd.concat([pd.concat([part1_sample1, part1_sample3]), part1_sample4])
f3 = pd.concat([pd.concat([part1_sample1, part1_sample2]), part1_sample4])
f4 = pd.concat([pd.concat([part1_sample1, part1_sample2]), part1_sample3])
# generate a random sample af the groups of 3
part2_sample1 = f1.sample(frac=0.0333, random_state=2)
part2_sample2 = f2.sample(frac=0.0333, random_state=3)
part2_sample3 = f3.sample(frac=0.0333, random_state=4)
part2_sample4 = f4.sample(frac=0.0333, random_state=5)
# concat every part of the sample, with the randomly generated sample from the group, that it was not a part of.
sample1 = pd.concat([part1_sample1, part2_sample1])
sample2 = pd.concat([part1_sample2, part2_sample2])
sample3 = pd.concat([part1_sample3, part2_sample3])
sample4 = pd.concat([part1_sample4, part2_sample4])
# extract each dataframe as csv file.
sample1.to_csv('sample1.csv')
sample2.to_csv('sample2.csv')
sample3.to_csv('sample3.csv')
sample4.to_csv('sample4.csv')

In [18]:
from pandas import ExcelWriter
# gem samples som excel filer
writer1 = ExcelWriter('sample1_17_08.xlsx')
sample1.to_excel(writer1,'Sheet1')
writer1.save()

writer2 = ExcelWriter('sample2_17_08.xlsx')
sample2.to_excel(writer2,'Sheet1')
writer2.save()

writer3 = ExcelWriter('sample3_17_08.xlsx')
sample3.to_excel(writer3,'Sheet1')
writer3.save()

writer4 = ExcelWriter('sample4_17_08.xlsx')
sample4.to_excel(writer4,'Sheet1')
writer4.save()